# BDD Shopping Cart: Complete Explanation

## 1. What is BDD?

Behavior-Driven Development bridges the gap between business requirements and technical implementation. It uses plain language to describe how software should behave.

## 2. The Workflow

```
Feature File (Gherkin) → Step Definitions (Python) → Production Code
```

- Business stakeholders write/review feature files
- Developers implement step definitions
- Step definitions test the actual application code

## 3. Key Components

### a) Feature File (`shopping_cart.feature`)
- Written in Gherkin language (Given-When-Then)
- Describes behavior from user's perspective
- Readable by non-technical stakeholders

### b) Step Definitions (`test_shopping_cart.py`)
- Python functions that execute each Gherkin step
- Use decorators (`@given`, `@when`, `@then`) to match steps
- `parsers.parse()` extracts values from step text

### c) Fixtures (`conftest.py`)
- Provide reusable setup code
- Create fresh instances for each test (isolation)
- The `cart` fixture gives each scenario a clean ShoppingCart

### d) Production Code (`shopping_cart.py`)
- The actual application being tested
- Implements business logic
- Uses Decimal for precise currency calculations

## 4. Why Decimal for Currency?

Floating-point numbers have precision issues:

**Problem:**
```python
0.1 + 0.2 = 0.30000000000000004  # Not exactly 0.3!
```

**Solution:**
```python
Decimal('0.1') + Decimal('0.2') = Decimal('0.3')  # Exact!
```

This prevents bugs in financial calculations.

## 5. Test Isolation

Each scenario gets a fresh cart via the fixture. This means:

- Scenario 1 doesn't affect Scenario 2
- Tests can run in any order
- Failures are easier to diagnose

## 6. The Feedback Loop

1. Write feature → Write step definitions → Implement code
2. Run tests to verify behavior
3. When **green**, everyone knows the feature works
4. When **red**, clear error messages show what broke

## 7. Benefits

- **Living documentation** (feature files)
- **Shared understanding** between business and tech
- **Automated verification** of requirements
- **Regression protection**

---

### Quick Reference: BDD Stack

| Layer | Technology | Purpose |
|-------|-----------|---------|
| Specification | Gherkin | Human-readable behavior descriptions |
| Test Framework | pytest-bdd | Connects Gherkin to Python |
| Test Organization | pytest fixtures | Manages test state and dependencies |
| Application | Python classes | Actual business logic |

In [1]:
!pip install pytest pytest-bdd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.0 MB/s eta 0:00:00


In [2]:
%%writefile shopping_cart.py
from decimal import Decimal
from typing import Dict

class ShoppingCart:
    """
    A shopping cart that manages products and calculates totals.

    This class represents the actual application code we're testing.
    It handles adding items and calculating the total price.
    """

    def __init__(self):
        """Initialize an empty shopping cart"""
        self._items: Dict[str, Decimal] = {}

    def clear(self):
        """Remove all items from the cart"""
        self._items.clear()

    def add_item(self, product_name: str, price: Decimal):
        """
        Add a product to the cart

        Args:
            product_name: Name of the product
            price: Price as a Decimal for precise currency handling
        """
        if product_name in self._items:
            # If item already exists, add to its price (quantity handling)
            self._items[product_name] += price
        else:
            self._items[product_name] = price

    def item_count(self) -> int:
        """Return the number of unique items in the cart"""
        return len(self._items)

    def get_total(self) -> Decimal:
        """Calculate and return the total price of all items"""
        return sum(self._items.values(), Decimal('0'))

    def get_items(self) -> Dict[str, Decimal]:
        """Return a copy of all items in the cart"""
        return self._items.copy()

Writing shopping_cart.py


In [3]:
!mkdir features

In [4]:
%%writefile features/shopping_cart.feature
Feature: Shopping Cart Management
  As a customer
  I want to manage items in my shopping cart
  So that I can purchase products

  Scenario: Adding items to an empty cart
    Given the shopping cart is empty
    When I add a product "Laptop" with price 999.99
    Then the cart should contain 1 item
    And the cart total should be 999.99

  Scenario: Adding multiple items
    Given the shopping cart is empty
    When I add a product "Laptop" with price 999.99
    And I add a product "Mouse" with price 25.50
    Then the cart should contain 2 items
    And the cart total should be 1025.49

  Scenario: Adding the same item twice
    Given the shopping cart is empty
    When I add a product "Laptop" with price 999.99
    And I add a product "Laptop" with price 999.99
    Then the cart should contain 1 item
    And the cart total should be 1999.98

Writing features/shopping_cart.feature


In [5]:
!mkdir tests

In [6]:
%%writefile tests/conftest.py
import pytest
import sys
sys.path.insert(0, '/content')  # Add root directory to path for imports

from shopping_cart import ShoppingCart

@pytest.fixture
def cart():
    """
    Fixture provides a fresh ShoppingCart instance for each scenario.

    This ensures test isolation - each scenario gets a clean cart,
    preventing tests from interfering with each other.

    Yields:
        ShoppingCart: A new, empty shopping cart instance
    """
    return ShoppingCart()

Writing tests/conftest.py


In [7]:
!mkdir tests/step_defs

In [8]:
%%writefile tests/step_defs/test_shopping_cart.py
import sys
sys.path.insert(0, '/content')

from pytest_bdd import scenarios, given, when, then, parsers
from decimal import Decimal

# Load all scenarios from the feature file
# This automatically creates test functions for each scenario
scenarios('../../features/shopping_cart.feature')

@given('the shopping cart is empty')
def empty_cart(cart):
    """
    Set up initial state: ensure cart has no items.

    We explicitly clear the cart to guarantee a known starting point.
    This prevents test failures due to leftover state from previous tests.

    Args:
        cart: The shopping cart fixture
    """
    cart.clear()
    assert cart.item_count() == 0, "Cart should be empty after clearing"

@when(parsers.parse('I add a product "{product_name}" with price {price:f}'))
def add_product(cart, product_name, price):
    """
    Perform the action: add a product to the cart.

    The parsers.parse decorator extracts:
    - product_name as a string (captured by quotes)
    - price as a float (converted using :f format)

    We convert to Decimal for precise currency handling, avoiding
    floating-point errors like 0.1 + 0.2 = 0.30000000000000004

    Args:
        cart: The shopping cart fixture
        product_name: Name of the product to add
        price: Price of the product as a float
    """
    cart.add_item(product_name, Decimal(str(price)))

@then(parsers.parse('the cart should contain {count:d} item'))
@then(parsers.parse('the cart should contain {count:d} items'))
def verify_item_count(cart, count):
    """
    Verify outcome: check the cart has expected number of items.

    The :d format specifier extracts count as an integer.
    We handle both singular "item" and plural "items" for natural language.

    Clear error messages help diagnose failures quickly by showing
    both expected and actual values.

    Args:
        cart: The shopping cart fixture
        count: Expected number of items
    """
    actual_count = cart.item_count()
    assert actual_count == count, \
        f"Expected {count} items, but cart has {actual_count}"

@then(parsers.parse('the cart total should be {expected_total:f}'))
def verify_cart_total(cart, expected_total):
    """
    Verify outcome: check the cart total matches expected value.

    Using Decimal ensures precise currency calculations.
    This prevents subtle bugs from floating-point arithmetic.

    Example: Without Decimal, 999.99 + 25.50 might not equal exactly 1025.49
    due to binary floating-point representation limitations.

    Args:
        cart: The shopping cart fixture
        expected_total: Expected total as a float
    """
    actual_total = cart.get_total()
    expected = Decimal(str(expected_total))
    assert actual_total == expected, \
        f"Expected total {expected}, but got {actual_total}"

Writing tests/step_defs/test_shopping_cart.py


In [9]:
# Run pytest with verbose output
!pytest tests/step_defs/test_shopping_cart.py -v --tb=short

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: bdd-8.1.0, anyio-4.11.0, typeguard-4.4.4, langsmith-0.4.35
collected 3 items                                                              

tests/step_defs/test_shopping_cart.py::test_adding_items_to_an_empty_cart PASSED [ 33%]
tests/step_defs/test_shopping_cart.py::test_adding_multiple_items PASSED [ 66%]
tests/step_defs/test_shopping_cart.py::test_adding_the_same_item_twice PASSED [100%]

============================== 3 passed in 0.25s ===============================


In [10]:
# Run with BDD-style output showing scenario steps
!pytest tests/step_defs/test_shopping_cart.py -v --gherkin-terminal-reporter

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: bdd-8.1.0, anyio-4.11.0, typeguard-4.4.4, langsmith-0.4.35
collected 3 items                                                              

tests/step_defs/test_shopping_cart.py::test_adding_items_to_an_empty_cart 
Feature: Shopping Cart Management
    Scenario: Adding items to an empty cart PASSED

tests/step_defs/test_shopping_cart.py::test_adding_multiple_items 
Feature: Shopping Cart Management
    Scenario: Adding multiple items PASSED

tests/step_defs/test_shopping_cart.py::test_adding_the_same_item_twice 
Feature: Shopping Cart Management
    Scenario: Adding the same item twice PASSED

============================== 3 passed in 0.03s ===============================
